In [ ]:
from models.decision_tree import DecisionTreeModel
from models.adaBoost import AdaBoostModel
from models.xg_boost import XGBoostModel
from models.gradient_boosting import GradientBoostingModel
from models.random_forest import RandomForestModel
from models.cat_boost import CatBoostModel
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, roc_auc_score
from metrics import cross_validate_with_resampling, Metrics
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import RandomOverSampler, SMOTE
import pandas as pd
import numpy as np

## Reading Data and Setting up Grid search parameters

In [ ]:
data = pd.read_csv('data/Telco-Customer-Churn-encoded-data-FE-Features-Selected.csv')
label = pd.read_csv('data/Telco-Customer-Churn-encoded-label.csv')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
data.head()

In [ ]:
label[:5]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Decision tree Grid search

In [ ]:
DecisionTreeModel_gs = DecisionTreeModel("DecisionTreeModel with grid search")
DT_best_params, DT_recempling = DecisionTreeModel_gs.gs_parameter_tune(X, y)


In [ ]:

DT_hyper_model = DecisionTreeModel("DecisionTreeModel with hyper parameters")
DT_hyper_model.hyper_parameter(DT_best_params)
if(DT_recempling):
    DT_hyper_model.fit_with_resampling(X_train, y_train)
else:
    DT_hyper_model.fit(X_train, y_train)

y_pred_hyper = DT_hyper_model.predict(X_test)

Metrics(y_test, y_pred_hyper).f1_score()

### XGBoost with Grid Search

In [ ]:
xgb_gs = XGBoostModel("XGBoostModel with grid search")
xgb_best_params, xgb_resempling = xgb_gs.gs_parameter_tune(X, y)

In [ ]:
xgb_hyper_model = XGBoostModel("XGBoostModel with hyper parameters")
xgb_hyper_model.hyper_parameter(xgb_best_params)
if(xgb_resempling):
    xgb_hyper_model.fit_with_resampling(X_train, y_train)
else:
    xgb_hyper_model.fit(X_train, y_train)

y_pred_hyper = xgb_hyper_model.predict(X_test)

Metrics(y_test, y_pred_hyper).f1_score()

### AdaBoost with Grid Search

In [ ]:
AdaBoostModel_gs = AdaBoostModel("AdaBoostModel with grid search")
ab_best_params, ab_resempling = AdaBoostModel_gs.gs_parameter_tune(X, y)

In [ ]:
ab_hyper_model = AdaBoostModel("AdaBoostModel with hyper parameters")
ab_hyper_model.hyper_parameter(ab_best_params)
if(ab_resempling):
    ab_hyper_model.fit_with_resampling(X_train, y_train)
else:
    ab_hyper_model.fit(X_train, y_train)

y_pred_hyper = ab_hyper_model.predict(X_test)

Metrics(y_test, y_pred_hyper).f1_score()

### Gradient Boosting with Grid Search

In [ ]:
GradientBoostingModel_gs = GradientBoostingModel("GradientBoostingModel with grid search")
gb_best_params, gb_resempling = GradientBoostingModel_gs.gs_parameter_tune(X, y)

In [ ]:
gb_hyper_model = GradientBoostingModel("GradientBoostingModel with hyper parameters")
gb_hyper_model.hyper_parameter(gb_best_params)
if(gb_resempling):
    gb_hyper_model.fit_with_resampling(X_train, y_train)
else:
    gb_hyper_model.fit(X_train, y_train)

y_pred_hyper = gb_hyper_model.predict(X_test)

Metrics(y_test, y_pred_hyper).f1_score()

In [ ]:
gb_hyper_model.save('gb_hyper_model.pkl')

In [ ]:
load_model = GradientBoostingModel()
load_model.load('gb_hyper_model.pkl')

In [ ]:
y_pred_load = load_model.predict(X_test)
Metrics(y_pred_load, y_test).f1_score()


### Random Forest Classifier with Grid Search

In [ ]:
RandomForestModel_gs = RandomForestModel("RandomForestModel with grid search")
rf_best_params, rf_resempling = RandomForestModel_gs.gs_parameter_tune(X, y)

In [ ]:
rf_hyper_model = RandomForestModel("RandomForestModel with hyper parameters")
rf_hyper_model.hyper_parameter(rf_best_params)
if(rf_resempling):
    rf_hyper_model.fit_with_resampling(X_train, y_train)
else:
    rf_hyper_model.fit(X_train, y_train)

y_pred_hyper = rf_hyper_model.predict(X_test)

Metrics(y_test, y_pred_hyper).f1_score()

### CatBoost Classifier with Grid Search

In [ ]:
CatBoost_gs = CatBoostModel("CatBoostModel with grid search")
cb_best_params, cb_resempling = CatBoost_gs.gs_parameter_tune(X, y)

In [ ]:
cb_hyper_model = CatBoostModel("CatBoostModel with hyper parameters")
cb_hyper_model.hyper_parameter(cb_best_params)
if(cb_resempling):
    cb_hyper_model.fit_with_resampling(X_train, y_train)
else:
    cb_hyper_model.fit(X_train, y_train)

y_pred_hyper = cb_hyper_model.predict(X_test)

Metrics(y_test, y_pred_hyper).f1_score()